## Using Numpy to Optimize our code

### What is numpy?


NumPy is the fundamental package for scientific computing with Python. It contains among other things:
* a powerful N-dimensional array object
* sophisticated (broadcasting) functions
* tools for integrating C/C++ and Fortran code
* useful linear algebra, Fourier transform, and random number capabilities

Besides its obvious scientific uses, NumPy can also be used as an efficient multi-dimensional container of generic data. [1]

In other words, it provide us with an object(the numpy N dimension array) where we can store our input data, labels and neural network parameters and easily do operations over them.

### How is it useful?
Since we can easily make operations over them we can take advantage of some matrix multiplication, addition and event apply a function over a numpy array. 

As a consequence instead of doing something like:

```python
    net_values = []

    '''Iterating thorugh each of the categories'''
    for category in range(0,10):
        weighted_sum = 0
        '''Calculating the weighted sum over all pixels on the image'''
        for i,pixel in enumerate(image):
            weighted_sum += weights[category][i]*pixel

        '''Appending to the net value array'''
        net_values.append(weighted_sum)
```
We can just take the dot product between the input and the weights:
```python

import numpy as np
weights_numpy = np.array(weights);
image_numpy = np.array(image)

'''Taking the dot product which is the same as the sum product 
of the weights with the pixels of an image'''
a = np.dot(weights_numpy,image_numpy)


```

### Lets try it. 

To prove how effective it is to use matrix multiplixation in numpy, lets do a simple experiment.

Lets run the the same likes of code with a single image. 


[1]:http://www.numpy.org/

In [1]:
'''Importing libraries'''
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cPickle, gzip
import numpy as np
import math
import time
%matplotlib inline

'''Reading the dataset'''
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()

train_set_data = train_set[0][:10000]
train_set_labels = train_set[1][:10000]

test_set_data = test_set[0][:1000]
test_set_labels = test_set[1][:1000]

valid_set_data = valid_set[0][:500]
valid_set_labels = valid_set[1][:500]

# Adding bias term to train_set and test_set
biases = np.array(([1] * 10000))
train_set_data  = np.column_stack((train_set_data,biases))

biases = np.array(([1] * 1000))
test_set_data  = np.column_stack((test_set_data,biases))

biases = np.array(([1] * 500))
valid_set_data  = np.column_stack((valid_set_data,biases))

In [2]:
'''Initializing the weight array using numpy random'''
categories = 10
inputdimension = 785
weigths = np.random.rand(categories, inputdimension)*0.0001
image = train_set_data[0]

#### Now lets see how long it takes to compute the net value per category in one single image.
#### Without Numpy

In [7]:
net_values = []
start_time = time.time()
'''Iterating thorugh each of the categories'''
for category in range(0,10):
    weighted_sum = 0
    '''Calculating the weighted sum over all pixels on the image'''
    for i,pixel in enumerate(image):
        weighted_sum += weigths[category][i]*pixel

    '''Appending to the net value array'''
    net_values.append(weighted_sum)
end_time = time.time()

print net_values

[0.0054354590276208337, 0.0050760159385886917, 0.0053035633007418462, 0.0058384878686459545, 0.0053606548933386577, 0.0051109461617070816, 0.0057464526078418419, 0.004775540291094396, 0.0059407176692748351, 0.0050490441279610219]


In [8]:
time_elpased_NoNumpy = end_time -start_time
print "Time Elapsed Without Numpy", time_elpased_NoNumpy

Time Elapsed Without Numpy 0.00961589813232


#### Woa, that was pretty fast now lets try it with Numpy

In [9]:
start_time = time.time()
'''Taking the weighted sum, which is simply the dot product of the weights with the image'''
a = np.dot(weigths,image)
end_time = time.time()

print a

[ 0.00543546  0.00507602  0.00530356  0.00583849  0.00536065  0.00511095
  0.00574645  0.00477554  0.00594072  0.00504904]


In [10]:
time_elpased_Numpy = end_time -start_time
print "Time Elapsed With Numpy", time_elpased_Numpy

Time Elapsed With Numpy 0.000249862670898


#### That was way faster lets check how many times faster

In [11]:
time_elpased_NoNumpy / time_elpased_Numpy

38.48473282442748

As you can see its more than an order of magitude faster than using a loop. This is due to the fact that numpy is optimized using C++ which makes it incredibly fast and efficient. The CPU is not looping though when we do the dot product it just does multiply and sum. 

Also, Numpy takes advantage of the all the cores in your computer which can run the computations in parallel. This is very important because it allow us to take adavantage of parallelism on multiple computers and even GPUs(Thanks Gamers). 

### Now Lets build a two layered(DEEP) network on numpy. 

#### BackPropagation
We have learned how to train the weights that exist between the input and the output using the **Learning Rule**. However, when dealing with more that one layer of weights we need to propagate the mistakes to train the weights on each layer. 
The layers that exist between the input and the output are called the the **Hidden Layers**.

This sounds pretty complicated and in reality it is. It was until the 70s when the method of propagating the error back to all layers got popularized. 

We can write down the derivation, but its better if we just see the **formula**. j units are the hidden layer units and k units are the output layer units:

$$ w_j(t+1) = w_j(t) + \alpha * \delta_j *x_j $$

We can notice that the only thing that will be different is the delta, or the change of the weight.


In the case of a hidden layer this delta would be different that the output delta since we do not have a teacher value of the correct result. 
In this formula $\delta_j$ is defined as:
$$\delta_j = y'_k \sum_{k}^{N} w_{jk}\delta_k$$

And $\delta_k$ as:
$$\delta_k = (teacher_k - y_k )$$ For a particular k unit, or in this category.

The parameters are defined as follow:

$\alpha$: Learning Rate

$y'_k$: The derivative of the output value of unit k, where k is the category. Pretty much softmax'(net_k)

$\sum_{k}^{N} w_{jk}\delta_k$: The sum of all the weights between the hidden and output layer mutiplied with their respective $\delta$ errors.


#### Why is this important. 

With the previous single layer neural network the weights leaned features about the pixels in the image. However, thse features were directly realted to the input value. There was no more to learn, which cause the neural network on fail in some cases. 

Now that we have more than one layer the neural network can learn more than input feautres. It is able to learn internal representations, which means that the weights in the layers in between the input and the output are feature that are recreated and learned by the neural network itself. 

Internal Representation its what have make neural networks so powerful.

#### Lets go over the steps of updating the weights on the Hidden Layers

* Step 1: Calculate the delta of the output layer
* Step 2: Calculate the delta of the hidden layer using the output layer delta
* Step 3: Update the hidden layer weights using the learning rule
* Step 4: Update the output layer weights using the learning rule

It is important to update the weights of the hidden layer first since they depend on the state of the output layer weights. 

### Now Lets Build the network

In [23]:
#Creating the hot key category label arrays
categoriesArray = []
for label in train_set_labels:
    tempArray = [0]*10
    tempArray[label] = 1
    categoriesArray.append(tempArray)

#### Let's Start with the Functions
#### Softmax or The Output function

In [17]:
'''Needed in order to apply it to a vector'''
def exponent(a):
    return math.exp(a)
'''Vectorizing the function in order to be applied to the vector'''
exponentVecorized = np.vectorize(exponent)

def softmax(a):
    net_k = exponentVecorized(a)
    denominator = np.sum(net_k)
    net_k = net_k/denominator
    return net_k

#### The hidden layer activation function:
In this case we are going to use a different activation function, in this case tanh or logistic regression works best.

In [18]:
def tanH(a):
    op1 = exponent(a)
    op2 = exponent(-a)
    return ((op1 - op2)/(op1 + op2))

'''The derivative'''
def tanHPrime(a):
    op1 = tanH(a)
    return (1-(op1*op1))

tanHVectorized = np.vectorize(tanH)
tanHPrimeVectorized = np.vectorize(tanHPrime)

#### Defining forward propagation using the dot product and the activation functions vertorized from numpy

In [19]:
def forwardPropagation(image, j_weigths, k_weights):

    #Starting at the j layer
    a_j = np.dot(j_weigths,image)
    net_j = tanHVectorized(a_j)
    net_j_prime = tanHPrimeVectorized(a_j)

    #On the forward layer
    a_k = np.dot(k_weights, net_j)
    #Softmax.
    net_k = softmax(a_k)
    
    return net_k, net_j, net_j_prime

### Defining BackPropagation

In [20]:
def backPropagation(image, net_k, net_jTuple , j_weigths, k_weights, index , learningRate):         
    net_j = net_jTuple[0]
    net_j_prime = net_jTuple[1]
    #BackPropagation
    #The output error
    delta_k = (categoriesArray[index] - net_k)

    #The hidden layer error
    delta_j = net_j_prime * np.sum((k_weights.T*delta_k).T,axis=0)
    
    #Calculating the update rule by mutiplyng the deltas by their inputs, The last part of the learning rule
    k_update = learningRate* np.outer(delta_k, net_j)  
    
    j_update = learningRate*np.outer(delta_j,image)  

    #Updating the weights.
    k_weights = k_weights + k_update  

    j_weigths = j_weigths + j_update   
    
    return  j_weigths,k_weights

### Defining a testing function, we need to check the accuracy of our network at every epoch 

In [21]:
def testing(set_data, labels, j_weights,k_weights , activationFunction):
    #Calculating error given the current weights at the epoch
    correct = 0.0
    #Iterating thorugh the data set
    for index, digit in enumerate(set_data):
        #Calculating the net output a over all the categories
         #Forward propagation.
            
        #Starting at the j layer
        a_j = np.dot(j_weights,digit)
        net_j = activationFunction(a_j)

        #On the forward layer
        a_k = np.dot(k_weights, net_j)
        net_k = softmax(a_k)
        
        #Obtaining the category with the max probability
        maximum = 0.0
        cat = 0
        for i,out in enumerate(net_k):
            if out > maximum:
                maximum = out
                cat = i
        #Checking if classification was accurate
        if(cat == labels[index] ):
            correct+=1.0
    #Calculating accuracy
    accuracy = correct/len(labels)
    return accuracy

### Now lets put it all together and run this network :D

In [ ]:
'''This define the dimension of the hidden layer, play around with it and check the accuracy'''
hiddenLayerSize = 50

'''Do not touch these numbers since they define the input and the output'''
imageSize = 785
outLayerCategories = 10
k_weights = np.random.rand(10,hiddenLayerSize) *0.0001
j_weigths = np.random.rand(hiddenLayerSize,785)*0.0001

learning_rate = 0.05

epochs = 10

for epoch in range(0,epochs):
    print "At Epoch: " , epoch

    #Adding stochasticity to the data set(shuffl)
    sample_indexes = np.arange(len(train_set_data))
    np.random.shuffle(sample_indexes)

    for index in sample_indexes:
        image = train_set_data[index]
        
        '''Calculating the output val from propagation'''
        retForward = forwardPropagation(image,j_weigths,k_weights)
        
        output = retForward[0]
        net_j = retForward[1]
        net_jPrime = retForward[2]
        
        net_jTuple = (net_j,net_jPrime)
        '''Updating the weights'''
        retWeight = backPropagation(image, output,net_jTuple , j_weigths,k_weights , index, learning_rate)
        
        '''Assigning the weights'''
        j_weigths = retWeight[0]
        k_weights = retWeight[1]
        
        
    #Calculating error given the current weights at the epoch for all sets
    valAccuracy =  testing(valid_set_data, valid_set_labels, j_weigths,k_weights , tanHVectorized)
    trainAccuracy =  testing(train_set_data, train_set_labels, j_weigths,k_weights , tanHVectorized)

    print "Train accuracy", trainAccuracy, "Validation accuracy", valAccuracy
        

At Epoch:  0
Train accuracy 0.9139 Validation accuracy 0.852
At Epoch:  1
Train accuracy 0.9245 Validation accuracy 0.89
At Epoch:  2
Train accuracy 0.9365 Validation accuracy 0.89
At Epoch:  3
Train accuracy 0.9328 Validation accuracy 0.876
At Epoch:  4
Train accuracy 0.9498 Validation accuracy 0.886
At Epoch:  5
Train accuracy 0.9593 Validation accuracy 0.916
At Epoch:  6
Train accuracy

### WOW Such Accuracy:
As you can see the performance of a single layer network was pretty good. This is due to the fact that the Neural Network was able to learn internal representations, which gives it more infomation to take decisions. 

Go to the next chapter in order to learn Neural Networks applied to **COMPUTER VISION**